In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import matplotlib.image as mpl
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
import shutil

In [2]:
base = 'C:\\Users\\thinh\\Downloads\\Datasetandrequirement\\data'

In [3]:
output_dir = 'C:\\Users\\thinh\\Downloads\\Datasetandrequirement\\split_data'
data_1_dir = os.path.join(output_dir, 'data_1')
data_2_dir = os.path.join(output_dir, 'data_2')
os.makedirs(data_1_dir, exist_ok=True)
os.makedirs(data_2_dir, exist_ok=True)

# Duyệt qua từng lớp
for class_name in os.listdir(base):
    class_dir = os.path.join(base, class_name)
    if os.path.isdir(class_dir):
        files = os.listdir(class_dir)
        
        # Chia thành hai phần 70-30
        train_files, test_files = train_test_split(files, test_size=0.3, random_state=42)
        
        # Tạo thư mục cho mỗi lớp
        train_class_dir = os.path.join(data_1_dir, class_name)
        test_class_dir = os.path.join(data_2_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
        
        # Di chuyển tệp vào thư mục tương ứng
        for file in train_files:
            shutil.copy(os.path.join(class_dir, file), os.path.join(train_class_dir, file))
        
        for file in test_files:
            shutil.copy(os.path.join(class_dir, file), os.path.join(test_class_dir, file))

print("Chia dữ liệu hoàn tất.")

Chia dữ liệu hoàn tất.


In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.3
)

In [5]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
train_generator = datagen.flow_from_directory(
    'C:\\Users\\thinh\\Downloads\\Datasetandrequirement\\split_data\\data_1',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='training',
    color_mode='rgb',
    seed=42,
    interpolation='nearest'
)

validation_generator = datagen.flow_from_directory(
    'C:\\Users\\thinh\\Downloads\\Datasetandrequirement\\split_data\\data_1',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='validation',
    color_mode='rgb',
    seed=42,
    interpolation='nearest'
)

Found 5631 images belonging to 4 classes.
Found 2411 images belonging to 4 classes.


In [7]:
class_name = train_generator.class_indices
class_name

{'cats': 0, 'chicken': 1, 'dogs': 2, 'squirrels': 3}

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32,(3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64,(3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128,(3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


c:\Users\thinh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
history = model.fit(
    train_generator,
    epochs=70,
    validation_data = validation_generator)
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=1)
print(f'Validation accuracy: {val_accuracy:.4f}')

Epoch 1/70


c:\Users\thinh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


176/176 ━━━━━━━━━━━━━━━━━━━━ 126s 694ms/step - accuracy: 0.4000 - loss: 1.3050 - val_accuracy: 0.4243 - val_loss: 1.2956
Epoch 2/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 74s 411ms/step - accuracy: 0.4571 - loss: 1.2569 - val_accuracy: 0.5620 - val_loss: 1.0468
Epoch 3/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 78s 439ms/step - accuracy: 0.5582 - loss: 1.0700 - val_accuracy: 0.5815 - val_loss: 1.0216
Epoch 4/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 59s 322ms/step - accuracy: 0.6081 - loss: 0.9768 - val_accuracy: 0.6333 - val_loss: 0.9106
Epoch 5/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 54s 298ms/step - accuracy: 0.6708 - loss: 0.8664 - val_accuracy: 0.6765 - val_loss: 0.8256
Epoch 6/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 51s 285ms/step - accuracy: 0.6982 - loss: 0.7890 - val_accuracy: 0.6574 - val_loss: 0.9138
Epoch 7/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 53s 297ms/step - accuracy: 0.7013 - loss: 0.7814 - val_accuracy: 0.7134 - val_loss: 0.7306
Epoch 8/70
176/176 ━━━━━━━━━━━━━━━━━━━━ 51s 285ms/step - accuracy: 0.7239 - loss: 0.7194 - va

In [10]:
model.save('model.h5')

In [11]:
loaded_model = models.load_model('model.h5')

In [12]:
train_generator_1 = datagen.flow_from_directory(
    'C:\\Users\\thinh\\Downloads\\Datasetandrequirement\\split_data\\data_2',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='training',
    color_mode='rgb',
    seed=42,
    interpolation='nearest'
)

validation_generator_1 = datagen.flow_from_directory(
    'C:\\Users\\thinh\\Downloads\\Datasetandrequirement\\split_data\\data_2',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='validation',
    color_mode='rgb',
    seed=42,
    interpolation='nearest'
)

Found 2416 images belonging to 4 classes.
Found 1033 images belonging to 4 classes.


In [13]:
loaded_model.compile(optimizer=Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [14]:
history = loaded_model.fit(
    train_generator_1,
    epochs=50,
    validation_data = validation_generator_1)
val_loss, val_accuracy = model.evaluate(validation_generator_1, verbose=1)
print(f'Validation accuracy: {val_accuracy:.4f}')

Epoch 1/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 47s 566ms/step - accuracy: 0.8076 - loss: 0.6640 - val_accuracy: 0.8209 - val_loss: 0.4925
Epoch 2/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 316ms/step - accuracy: 0.8395 - loss: 0.4116 - val_accuracy: 0.8248 - val_loss: 0.5138
Epoch 3/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 312ms/step - accuracy: 0.8547 - loss: 0.4096 - val_accuracy: 0.8267 - val_loss: 0.4821
Epoch 4/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 312ms/step - accuracy: 0.8509 - loss: 0.3880 - val_accuracy: 0.8364 - val_loss: 0.4886
Epoch 5/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 320ms/step - accuracy: 0.8725 - loss: 0.3456 - val_accuracy: 0.8316 - val_loss: 0.5230
Epoch 6/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 314ms/step - accuracy: 0.8809 - loss: 0.3395 - val_accuracy: 0.8296 - val_loss: 0.5009
Epoch 7/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 316ms/step - accuracy: 0.8939 - loss: 0.3067 - val_accuracy: 0.8412 - val_loss: 0.4724
Epoch 8/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 25s 311ms/step - accuracy: 0.9013 - loss: 0.2801 - val_accu